In [1]:
import os
import time
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.random import set_seed
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import Sequential
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

In [2]:
# check if GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


sgd = SGD(lr=0.01, momentum=0.9, decay=0.0001, nesterov=True)

In [3]:
path = 'C:/Users/promm/Documents/S2-classification/Data/'

# load train, validation and test data
train = pd.read_csv(os.path.join(path, 'train.csv'))
val = pd.read_csv(os.path.join(path, 'val.csv'))
test = pd.read_csv(os.path.join(path, 'test.csv'))

In [4]:
# keep columns starting with ndvi in the name, also ID and specie columns
train = train.filter(regex='b0|ID|specie')
val = val.filter(regex='b0|ID|specie')
test = test.filter(regex='b0|ID|specie')

In [5]:
# create a dictionary to convert labels to integers
labels = train['specie'].unique()
labels_dict = dict(zip(labels, range(len(labels))))

In [6]:
# convert labels to integers
train['specie'] = train['specie'].map(labels_dict)
val['specie'] = val['specie'].map(labels_dict)
test['specie'] = test['specie'].map(labels_dict)

In [7]:
train_labels = to_categorical(train['specie'])
val_labels = to_categorical(val['specie'])
test_labels_expanded = to_categorical(test['specie'])
test_labels = test['specie']
train = train.drop(['ID','specie'], axis=1)
val = val.drop(['ID','specie'], axis=1)
ids = test['ID']
test = test.drop(['ID','specie'], axis=1)


In [8]:
mpath = 'C:/Users/promm/Documents/S2-classification/models'

In [9]:
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = 10,10


In [10]:
def get_model(ts=train.shape[1], tl=train_labels.shape[1], conv = False, conv_layers=[], dense_layers=[], dropout=0.15, lr=0.01):
    """


    Args:
        conv (bool, optional): _description_. Defaults to False.
        conv_layers (list, optional): _description_. Defaults to [].
        dense_layers (list, optional): _description_. Defaults to [].
        dropout (float, optional): _description_. Defaults to 0.5.
        lr (float, optional): _description_. Defaults to 0.001.
        batch_size (int, optional): _description_. Defaults to 32.
        epochs (int, optional): _description_. Defaults to 100.
        patience (int, optional): _description_. Defaults to 10.
        verbose (int, optional): _description_. Defaults to 1.
    """

    model = Sequential()
    if conv:
        model.add(Input(shape=(ts,1)))

        for layer in conv_layers:
            model.add(Conv1D(layer, 3, activation='relu'))
            model.add(BatchNormalization())
            model.add(MaxPooling1D(2))
        model.add(Flatten())
    else:
        model.add(Input(shape=(ts,)))
    for layer in dense_layers:
        model.add(Dense(layer, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))
    model.add(Dense(tl, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [11]:
conv = [False, False, False, False, False, False, False, False, False, False, False, False, False, False,
        True, True, True, True, True, True, True, True, True, True, True, True]
conv_layers = [[], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [64], [64], [64], [32], [32], [32], [64,32], [64,32], [64,32], [32,16], [32,16], [32,16],]
dense_layers = [[64], [128], [256], [512,256], [512,128], [128,64], [128,32], [256,128], [256,64], [256,32], [256,256,128], [128,64,32], [256,128,64], [256,128,32],
                [64], [64,32], [128,64],[64], [64,32], [128,64],[64], [64,32], [128,64],[64], [64,32], [128,64]]

In [12]:
kind = 'band'

flag = True

for i in range(len(conv)):

    if flag:
        if conv[i]:
            # do things
            train = train.to_numpy().reshape(train.shape[0], train.shape[1],1)
            val = val.to_numpy().reshape(val.shape[0], val.shape[1],1)
            test = test.to_numpy().reshape(test.shape[0], test.shape[1],1)
            flag = False


    model = get_model(conv=conv[i], conv_layers=conv_layers[i], dense_layers=dense_layers[i])
    model.summary()
    model_name = f'model_{kind}_{i}.h5'

    np.random.seed(123)
    random.seed(123)
    set_seed(123)

    model_checkpoint_callback = ModelCheckpoint(os.path.join(mpath, model_name), monitor='val_loss', mode='min', save_best_only=True)
    early_stopping_callback = EarlyStopping(monitor='val_loss', patience=40, mode='min')
    reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, mode='min')

    start = time.time()
    history = model.fit(train, train_labels, epochs=600, batch_size=8, validation_data=(val, val_labels),
                        callbacks=[model_checkpoint_callback, early_stopping_callback, reduce_lr_callback], verbose = 0)
    end = time.time()
    print(f'{end - start} seconds')

    test_loss, test_acc = model.evaluate(test, test_labels_expanded)
    y_pred = model.predict(test)
    # get the index of the max probability
    y_pred = np.argmax(y_pred, axis=1)

    # plot loss and accuracy
    plt.plot(history.history['accuracy'], label='train_acc')
    plt.plot(history.history['val_accuracy'], label='val_acc')
    plt.legend()
    plt.savefig(os.path.join(mpath, f'model_history_{kind}_{i}.png'), dpi=300)
    plt.close()

    # confusion matrix
    cm = confusion_matrix(test_labels, y_pred)
    # plot confusion matrix
    plt.imshow(cm, cmap='CMRmap_r')
    plt.xlabel("Predicted labels")
    plt.ylabel("True labels")
    # add ticks values from labels
    tick_marks = np.arange(len(labels))
    # rotate x ticks to 90 degrees
    plt.xticks(tick_marks, labels, rotation=90)
    plt.yticks(tick_marks, labels)
    plt.title('Confusion matrix ')
    plt.colorbar()
    plt.savefig(os.path.join(mpath, f'cm_{kind}_{i}.png'), dpi=300)
    plt.close()

    # dataframe from y_pred, test_labels
    df = pd.DataFrame({'FID':ids, 'y_pred': y_pred, 'y_obs': test_labels})
    df.to_csv(os.path.join(mpath, f'pred_{kind}_{i}.csv'), index=False)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                3392      
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                1040      
                                                                 
Total params: 4,688
Trainable params: 4,560
Non-trainable params: 128
_________________________________________________________________
85.43986821174622 seconds
15/15 [==============================] - 0s 1ms/step
Model: "sequential_1"
_________________